In [59]:
from dqt import DBUtils, get_fut_min, get_fut_taq
import pandas as pd


In [178]:
dbu = DBUtils()

all_varieties = ['A', 'AG', 'AL', 'AU', 'AP', 'BU', 'B', 'C', 'CF', 'CJ', 'CS', 'CU', 'CY', 'EG', 'FG', 'HC', 'I', 'J',
                'JD', 'JM', 'L', 'M', 'MA', 'NI', 'NR', 'OI', 'P', 'PB', 'PP', 'RB', 'RM', 'RU', 'SC', 'SP', 'SR', 'T',
                'TA', 'TF', 'TS', 'UR', 'V', 'Y', 'ZC', 'ZN']



all_trading_days = dbu.get_trading_day()
first_trading_day = all_trading_days[0]
print('first_trading_day: {}'.format(first_trading_day))
tick_data_first_day = '2014-01-01'

first_trading_day: 1990-12-19 00:00:00


In [20]:
all_trading_days

DatetimeIndex(['1990-12-19', '1990-12-20', '1990-12-21', '1990-12-24',
               '1990-12-25', '1990-12-26', '1990-12-27', '1990-12-28',
               '1990-12-31', '1991-01-02',
               ...
               '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27',
               '2020-02-28', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06'],
              dtype='datetime64[ns]', name='tradingDay', length=7146, freq=None)

In [179]:
missing_trading_day_data_dict = dict()
for variety in all_varieties:#遍历所有品种
    print('variety: {}'.format(variety))
    main_daily_df = dbu.get_main_daily(variety=variety, start_date=tick_data_first_day)
    #print('main_daily_df: \n', main_daily_df)
    if main_daily_df.empty:
        missing_trading_day_data_dict[variety] = 'missing all data'
        continue
    #print('main_df: ', main_df)
    symbols_start_end_day_df = main_daily_df.groupby('symbol').agg({'date':['first', 'last']})
    print('symbols_start_end_day_df: \n', symbols_start_end_day_df)
    variety_first_last_day = symbols_start_end_day_df.agg({('date','first'):'min', ('date','last'):'max'})
    variety_first_day = variety_first_last_day[('date','first')]
    print('variety_first_day: ', variety_first_day)
    variety_last_day = variety_first_last_day[('date','last')]
    print('variety_last_day: {}'.format(variety_last_day))
    variety_all_trading_days = pd.Series(range(len(all_trading_days)), index = all_trading_days)[variety_first_day:variety_last_day].index
    
    #print('symbols_start_end_day_df: \n', symbols_start_end_day_df)
    #print('symbols_start_end_day_df.values: \n', symbols_start_end_day_df.values)
    #print('symbols_start_end_day_df.columns: \n', symbols_start_end_day_df.columns)
    #print('symbols_start_end_day_df.index: \n', symbols_start_end_day_df.index)
    variety_taq_days_list = list()
    
    for idx, row in symbols_start_end_day_df.iterrows():#遍历这个品种的所有合约
        """
        print('idx: ', idx)
        print('row: \n', row)
        print('type(row): ', type(row))
        print('row.index: ', row.index)
        print('row[date]: \n', row['date'])
        print('row[(date, first)]: \n', row[('date','first')])
        print('row.values: \n', row.values)
        """
        symbol_first_day = row[('date', 'first')].date()
        symbol_last_day = row[('date', 'last')].date()
        #print('symbol_first_day: ', symbol_first_day)
        #print('symbol_last_day: ', symbol_last_day)
        try:
            taq_df = get_fut_taq(row.name, start_date = symbol_first_day, end_date = symbol_last_day)
        except:
            print('{} missing TAQ data'.format(row.name))
            try:
                missing_trading_day_data_dict[variety].append('{} missing TAQ data, get_fut_taq function error'.format(row.name))
            except:
                missing_trading_day_data_dict[variety] = list()
                missing_trading_day_data_dict[variety].append('{} missing TAQ data, get_fut_taq function error'.format(row.name))
            continue
        symbol_taq_days_list = list(taq_df.groupby(taq_df.index).groups.keys())
        del taq_df
        #print('len(symbol_taq_days_list): ', len(symbol_taq_days_list))
        variety_taq_days_list += symbol_taq_days_list
        #print('len(variety_taq_days_list): ', len(variety_taq_days_list))

    #print('variety_taq_days_list: ', variety_taq_days_list)
    variety_taq_days_dtidx = pd.DatetimeIndex(variety_taq_days_list)
    del variety_taq_days_list
    #print('variety_taq_days_dtidx: ', variety_taq_days_dtidx)
    variety_trading_day_diff = variety_all_trading_days.difference(variety_taq_days_dtidx)
    if variety_trading_day_diff.empty:
        continue
    del variety_taq_days_dtidx
    #print('trading_day_diff: ', trading_day_diff)
    try:
        missing_trading_day_data_dict[variety].append(variety_trading_day_diff)
    except:
        missing_trading_day_data_dict[variety] = variety_trading_day_diff
    del variety_trading_day_diff
    
print('####finished checking all varieties####')

    

    

variety: A
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
A1409  2014-01-02 2014-03-24
A1501  2014-03-25 2014-11-20
A1505  2014-11-21 2015-04-03
A1509  2015-04-07 2015-06-11
A1601  2015-06-12 2015-11-02
A1605  2015-11-03 2016-03-04
A1609  2016-03-07 2016-07-11
A1701  2016-07-12 2016-11-24
A1705  2016-11-25 2017-03-21
A1709  2017-03-22 2017-07-31
A1801  2017-08-01 2017-12-06
A1805  2017-12-07 2018-04-02
A1809  2018-04-03 2018-07-25
A1901  2018-07-26 2018-12-19
A1905  2018-12-20 2019-04-17
A1909  2019-04-18 2019-08-07
A2001  2019-08-08 2019-09-09
variety_first_day:  2014-01-02 00:00:00
variety_last_day: 2019-09-09 00:00:00
variety: AG
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
AG1406 2014-01-02 2014-05-07
AG1412 2014-05-08 2014-11-03
AG1506 2014-11-04 2015-04-30
AG1512 2015-05-04 2015-11-13
AG1606 2015-11-16 2016-05-03
AG1612 2016-05-04 2016-11-09
AG

variety: CY
variety: EG
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
EG1906 2018-12-10 2019-04-26
EG1909 2019-04-29 2019-07-31
EG2001 2019-08-01 2019-09-09
variety_first_day:  2018-12-10 00:00:00
variety_last_day: 2019-09-09 00:00:00
variety: FG
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
FG1405 2014-01-02 2014-03-14
FG1409 2014-03-17 2014-07-08
FG1501 2014-07-09 2014-11-12
FG1506 2014-11-13 2015-04-02
FG1509 2015-04-03 2015-08-06
FG1601 2015-08-07 2015-11-25
FG1605 2015-11-26 2016-04-05
FG1609 2016-04-06 2016-08-10
FG1701 2016-08-11 2016-11-30
FG1705 2016-12-01 2017-04-07
FG1709 2017-04-10 2017-08-08
FG1801 2017-08-09 2017-11-24
FG1805 2017-11-27 2018-04-03
FG1809 2018-04-04 2018-08-06
FG1901 2018-08-07 2018-12-10
FG1905 2018-12-11 2019-04-09
FG1909 2019-04-10 2019-08-06
FG2001 2019-08-07 2019-09-09
variety_first_day:  2014-01-02 00:00:00
variety

variety: P
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
P1405  2014-01-02 2014-02-26
P1409  2014-02-27 2014-06-17
P1501  2014-06-18 2014-11-06
P1505  2014-11-07 2015-03-06
P1509  2015-03-09 2015-06-05
P1601  2015-06-08 2015-11-26
P1605  2015-11-27 2016-03-14
P1609  2016-03-15 2016-07-19
P1701  2016-07-20 2016-11-25
P1705  2016-11-28 2017-03-20
P1709  2017-03-21 2017-07-24
P1801  2017-07-25 2017-11-24
P1805  2017-11-27 2018-03-26
P1809  2018-03-27 2018-07-26
P1901  2018-07-27 2018-12-04
P1905  2018-12-05 2019-04-10
P1909  2019-04-11 2019-08-08
P2001  2019-08-09 2019-09-09
variety_first_day:  2014-01-02 00:00:00
variety_last_day: 2019-09-09 00:00:00
variety: PB
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
PB1402 2014-01-02 2014-01-21
PB1403 2014-01-22 2014-02-13
PB1404 2014-02-14 2014-03-17
PB1405 2014-03-18 2014-04-25
PB1406 2014-04-28 2014-05-19
PB

variety: TA
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
TA1405 2014-01-02 2014-03-07
TA1409 2014-03-10 2014-08-04
TA1501 2014-08-05 2014-11-17
TA1505 2014-11-18 2015-03-27
TA1509 2015-03-30 2015-08-03
TA1601 2015-08-04 2015-11-20
TA1605 2015-11-23 2016-03-25
TA1609 2016-03-28 2016-08-02
TA1701 2016-08-03 2016-11-11
TA1705 2016-11-14 2017-04-05
TA1709 2017-04-06 2017-08-01
TA1801 2017-08-02 2017-11-21
TA1805 2017-11-22 2018-03-22
TA1809 2018-03-23 2018-08-02
TA1901 2018-08-03 2018-11-27
TA1905 2018-11-28 2019-04-03
TA1909 2019-04-04 2019-08-12
TA2001 2019-08-13 2019-09-09
variety_first_day:  2014-01-02 00:00:00
variety_last_day: 2019-09-09 00:00:00
variety: TF
symbols_start_end_day_df: 
              date           
            first       last
symbol                      
TF1403 2014-01-02 2014-02-18
TF1406 2014-02-19 2014-05-13
TF1409 2014-05-14 2014-08-12
TF1412 2014-08-13 2014-10-31
TF1503 2014-11-03 2015-02-02
T

In [180]:
missing_trading_day_data_dict

{'CY': 'missing all data',
 'NR': ['NR2002 missing TAQ data, get_fut_taq function error',
  DatetimeIndex(['2019-08-12', '2019-08-13', '2019-08-14', '2019-08-15',
                 '2019-08-16', '2019-08-19', '2019-08-20', '2019-08-21',
                 '2019-08-22', '2019-08-23', '2019-08-26', '2019-08-27',
                 '2019-08-28', '2019-08-29', '2019-08-30', '2019-09-02',
                 '2019-09-03', '2019-09-04', '2019-09-05', '2019-09-06'],
                dtype='datetime64[ns]', freq=None)],
 'SC': ['SC1809 missing TAQ data, get_fut_taq function error',
  'SC1812 missing TAQ data, get_fut_taq function error',
  'SC1901 missing TAQ data, get_fut_taq function error',
  'SC1903 missing TAQ data, get_fut_taq function error',
  'SC1904 missing TAQ data, get_fut_taq function error',
  'SC1905 missing TAQ data, get_fut_taq function error',
  'SC1906 missing TAQ data, get_fut_taq function error',
  'SC1907 missing TAQ data, get_fut_taq function error',
  'SC1908 missing TAQ data, g

In [ ]:
missing_trading_day_data_dict

In [174]:
main_daily_df = dbu.get_main_daily(variety='TF', start_date='2019-09-02')
main_daily_df

,adj_factor,date,symbol,open,high,low,close,volume,open_interest,turnover,...,settle_ret,amplitude,left_days,interday_ret,high_ret,low_ret,open_ret,intraday_ret,trade_symbol,projected_value
0,0.982766,2019-09-02,TF1912,100.010,100.050,99.915,99.925,9653.0,26352.0,964952.690,...,-0.00075,0.001,102,0.000100,-0.000699,-0.000350,-0.000999,-0.000850,TF1912,27037.460082
1,0.982766,2019-09-03,TF1912,99.900,99.975,99.885,99.885,5860.0,26586.0,585589.685,...,-0.00050,0.001,101,-0.000250,-0.000750,-0.000300,-0.001100,-0.000150,TF1912,27120.196159
2,0.982766,2019-09-04,TF1912,99.870,100.000,99.870,99.985,9707.0,27443.0,970242.950,...,0.00080,0.001,100,-0.000150,0.000250,-0.000150,-0.000300,0.001151,TF1912,28085.167293
3,0.982766,2019-09-05,TF1912,100.160,100.185,100.085,100.130,9459.0,28023.0,947235.105,...,0.00165,0.001,99,0.001750,0.001850,0.002153,0.002904,-0.000300,TF1912,28644.397431
4,0.982766,2019-09-06,TF1912,100.145,100.275,100.100,100.195,11475.0,29381.0,1149811.850,...,0.00065,0.002,98,0.000150,0.000898,0.000150,-0.000150,0.000499,TF1912,30248.306001
5,0.982766,2019-09-09,TF1912,100.275,100.275,100.110,100.135,8227.0,29710.0,823904.555,...,-0.00075,0.002,95,0.000798,0.000000,0.000100,0.001298,-0.001396,TF1912,30397.688363


In [177]:
get_fut_taq('TF1912', start_date='2019-09-04', end_date='2019-09-05')

,datetime,last,volume,turnover,oi,a1,a1v,b1,b1v,symbol
trading_date,,,,,,,,,,
2019-09-05,2019-09-05 09:14:00.200,100.160,7,7011200,27446,100.230,1,100.160,8,TF1912
2019-09-05,2019-09-05 09:15:00.200,100.160,7,7011200,27446,100.220,8,100.180,10,TF1912
2019-09-05,2019-09-05 09:15:00.700,100.180,23,23039700,27460,100.185,3,100.170,4,TF1912
2019-09-05,2019-09-05 09:15:01.200,100.165,36,36061900,27468,100.165,2,100.150,3,TF1912
2019-09-05,2019-09-05 09:15:01.700,100.160,91,91149950,27499,100.140,1,100.100,32,TF1912
2019-09-05,2019-09-05 09:15:02.200,100.140,124,124200400,27502,100.150,5,100.130,3,TF1912
2019-09-05,2019-09-05 09:15:02.700,100.120,133,133211750,27503,100.135,10,100.130,1,TF1912
2019-09-05,2019-09-05 09:15:03.200,100.140,135,135214550,27501,100.140,3,100.135,3,TF1912
2019-09-05,2019-09-05 09:15:03.700,100.145,141,141223000,27496,100.145,1,100.140,48,TF1912
